<a href="https://colab.research.google.com/github/blueCormorant/MCTest-Augmented-/blob/master/Multiple_Choice_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.1 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM
import numpy as np

In [ ]:
prompt = "John told Mary to fire Alice, and he told Bill to hire her. Who did John tell him to hire?\n\n\n"
options = ["John", "Mary", "Alice", "Bill"]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
inputs = tokenizer([prompt], return_tensors="pt")

In [ ]:
outputs = model.generate(
    **inputs,
    max_new_tokens=1,
    return_dict_in_generate=True,
    output_scores=True,
    num_return_sequences=5,
    do_sample=True,
    prefix_allowed_tokens_fn=lambda batch_id, sent: [tokenizer.encode(token) for token in options]
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)

In [ ]:
input_length = inputs.input_ids.shape[1]

In [ ]:
generated_tokens = outputs.sequences[:, input_length:]

In [ ]:
print(f"| token | token string | logits | probability |")
for i in range(len(list(zip(generated_tokens, transition_scores)))):
    for tok, score in zip(generated_tokens[i], transition_scores[i]):
        print(f"| {tok:5d} | {tokenizer.decode(tok):12s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}      |")

| token | token string | logits | probability |
| 44484 | Alice        | -1.200 | 30.11%      |
| 24119 | Mary         | -0.790 | 45.37%      |
| 44484 | Alice        | -1.200 | 30.11%      |
| 24119 | Mary         | -0.790 | 45.37%      |
| 17798 | Bill         | -2.219 | 10.87%      |


In [ ]:
outputs.sequences

tensor([[ 7554,  1297,  5335,   284,  2046, 14862,    11,   290,   339,  1297,
          3941,   284, 11078,   607,    13,  5338,   750,  1757,  1560,   683,
           284, 11078,    30,   628,   198, 44484],
        [ 7554,  1297,  5335,   284,  2046, 14862,    11,   290,   339,  1297,
          3941,   284, 11078,   607,    13,  5338,   750,  1757,  1560,   683,
           284, 11078,    30,   628,   198, 24119],
        [ 7554,  1297,  5335,   284,  2046, 14862,    11,   290,   339,  1297,
          3941,   284, 11078,   607,    13,  5338,   750,  1757,  1560,   683,
           284, 11078,    30,   628,   198, 44484],
        [ 7554,  1297,  5335,   284,  2046, 14862,    11,   290,   339,  1297,
          3941,   284, 11078,   607,    13,  5338,   750,  1757,  1560,   683,
           284, 11078,    30,   628,   198, 24119],
        [ 7554,  1297,  5335,   284,  2046, 14862,    11,   290,   339,  1297,
          3941,   284, 11078,   607,    13,  5338,   750,  1757,  1560,   683,
  